- Transformer는 encoder --> decoder
- encoder를 이용해서 만든 언어모델 BERT : 감성분류, 스팸, 개체명인식, 유사도 측정 --> BERT는 이해, 분류를 잘함
- decoder를 이용해서 만든 언어모델 GPT : 언어 추론 요약, QA챗봇 --> GPT는 생성을 잘함

KoBERT를 이용해서 빈 단어 맞추기

# colab에서 해보기!

In [1]:
# BERT가 잘하는 것 : 분류, 빈칸 추론, 추측, 문장 임베딩 (생성XX)


In [ ]:
# %pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer": Expected package name at the start of dependency specifier
    'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer
    ^
Hint: = is not a valid operator. Did you mean == ?
'subdirectory'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [1]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

ModuleNotFoundError: No module named 'kobert_tokenizer'

In [ ]:
%pip install transformers

In [4]:
# mask를 씌우는 모델

from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('skt/kobert-base-v1')
model.eval()

Some weights of BertForMaskedLM were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [5]:
# 1. 상식 추론
import torch
text = "한국의 수도는 [MASK]입니다."
input_ids = tokenizer.encode(text, return_tensors='pt')
print(tokenizer.mask_token_id, input_ids)
# 출력 : (4, tensor([[   2, 4962, 2874, 5760,    4,  517, 7139,   54,    3]]))
# tokenizer.mask_token_id ==> 4

# mask_token_id 위치 찾기
# torch.where(input_ids == tokenizer.mask_token_id) ==> (tensor([0]), tensor([4]))
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]


# 마스크위치에서 가까운 상위 5개 추출
# 추론
with torch.no_grad():
    output = model(input_ids)
    print(output) # ==> logits 가 정답임
    predictions = output.logits # 정렬해서 예측함..???
print(predictions.shape) # ==> 출력: torch.Size([1, 9, 8002]) ==> 9: 문장길이, 8002 : 임베딩갯수(차원)
print('===============')

print(predictions[0, mask_token_index, :])
print('===============')
masked_prediciton = predictions[0, mask_token_index, :].topk(5) # 상위 5개
print(masked_prediciton)

for i, index_t in enumerate(masked_prediciton.indices[0]):
    index = index_t.item()
    print(tokenizer.decode([index]))

# 결과 좋지않음.
# skt/kobert-base-v1 단어맞추는 출력 헤더가 붙어 있어서 MASK를 맞추는게 가능함.
# kobert 모델에 가중치가 없는 것 같음.
# skt/kobert-base-v1 모델은 Masked Language Model(MLM)으로 파인튜닝된 적이 없어서 정확한 마스크 예측 능력이 떨어짐

NameError: name 'tokenizer' is not defined

### huggingface의 klue/bert-base 이용
- https://huggingface.co/klue/bert-base#how-to-get-started-with-the-model

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
name="klue/bert-base"
model = AutoModelForMaskedLM.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name)


# AutoModelForMAskedLM 이라서 추론이 가능함?
# 그냥 bert모델은 추론할 수 있는 헤드가 없음..?

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 1. 상식 추론
import torch
text = "대한민국의 수도는 [MASK]입니다."
inputs = tokenizer(text,return_tensors='pt')
mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
# 추론
with torch.no_grad():
  outputs = model(**inputs)
predictions = outputs.logits
print(predictions[0,mask_token_index,:])
masked_prediction = predictions[0,mask_token_index,:].topk(5)
for i, index_t in enumerate(masked_prediction.indices[0]):
  index = index_t.item()
  print(tokenizer.decode([index]))


tensor([[-5.1762,  6.2289, -4.8118,  ..., -5.0996, -3.0230, -3.9644]])
서울
광화문
부산
평양
인천
